In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import keras.backend as K
# %pip install tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint

In [17]:
df = pd.read_csv('../../Dataset/cleaned_used_cars_v1.csv')

In [18]:
df.shape

(207735, 12)

In [19]:
# dummies = pd.get_dummies(df.drop(['dateCrawled','lastSeen','dateCreated','name'], axis=1), columns=['seller', 'offerType','abtest','vehicleType','gearbox','model','fuelType','brand','notRepairedDamage'])
# df_preprocessed = pd.concat([df.drop(['price','index','dateCrawled','lastSeen','yearOfRegistration','monthOfRegistration','nrOfPictures','postalCode',
#              'name','seller','offerType','abtest','vehicleType','gearbox','model','fuelType','brand','notRepairedDamage','dateCreated'], axis=1),
#                              dummies], axis=1)
onehot_columns = ['abtest','vehicleType','gearbox','model','fuelType','brand','notRepairedDamage']
onehot_df = df[onehot_columns]
onehot_df = pd.get_dummies(onehot_df, columns = onehot_columns)
score_onehot_drop = df.drop(onehot_columns, axis = 1)
score_onehot = pd.concat([score_onehot_drop, onehot_df], axis = 1)
df_preprocessed = score_onehot

In [20]:
df_preprocessed.shape

(207735, 298)

In [21]:
df_preprocessed

,price,yearOfRegistration,powerPS,kilometer,monthOfRegistration,abtest_control,abtest_test,vehicleType_compact car,vehicleType_convertible,vehicleType_other,...,brand_skoda,brand_smart,brand_subaru,brand_suzuki,brand_toyota,brand_trabant,brand_volkswagen,brand_volvo,notRepairedDamage_no,notRepairedDamage_yes
0,1500,2001,75,150000,6,0,1,1,0,0,...,0,0,0,0,0,0,1,0,1,0
1,3600,2008,69,90000,7,0,1,1,0,0,...,1,0,0,0,0,0,0,0,1,0
2,650,1995,102,150000,10,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,2200,2004,109,150000,8,0,1,0,1,0,...,0,0,0,0,0,0,0,0,1,0
4,0,1980,50,40000,7,0,1,0,0,0,...,0,0,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207730,7900,2010,140,150000,7,0,1,0,0,0,...,0,0,0,0,0,0,1,0,1,0
207731,3999,2005,3,150000,5,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
207732,3200,2004,225,150000,5,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
207733,1199,2000,101,125000,3,0,1,0,1,0,...,0,1,0,0,0,0,0,0,1,0


In [22]:
def r2(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [23]:
X = df_preprocessed.astype(float).to_numpy()
y = df['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
# model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
# model.add(Dropout(0.3))
model.add(Dense(16, activation='relu'))
# model.add(Dropout(0.1))
model.add(Dense(1))

print('--------------------Compile------------------------------------')
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001, amsgrad=True),
              loss='mean_squared_error',
              metrics=['mae', r2])

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# early_stop = EarlyStopping(monitor='val_loss', patience=100)

# checkpoint = ModelCheckpoint(filepath='best_weights_amsgrad.h5', 
#                              monitor='val_loss', 
#                              save_best_only=True, 
#                              mode='min', 
#                              verbose=1)

print('--------------------Fit------------------------------------')
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=50, epochs=100)

print('--------------------Evaluate------------------------------------')
score = model.evaluate(X_test, y_test)
print(f'Deep Learning Regression Mean Absolute Error: {score[1]:.2f}')
print(f'Deep Learning Regression R-squared Score: {score[2]:.2f}')

--------------------Compile------------------------------------
--------------------Fit------------------------------------
Epoch 1/100
2659/2659 [==============================] - 13s 4ms/step - loss: 46622732288.0000 - mae: 2160.0071 - r2: 0.6932 - val_loss: 160310312960.0000 - val_mae: 2776.5588 - val_r2: 0.9396
Epoch 2/100
2659/2659 [==============================] - 11s 4ms/step - loss: 40232247296.0000 - mae: 918.9005 - r2: 0.9560 - val_loss: 138021093376.0000 - val_mae: 2479.6597 - val_r2: 0.9607
Epoch 3/100
2659/2659 [==============================] - 11s 4ms/step - loss: 35405320192.0000 - mae: 804.3495 - r2: 0.9681 - val_loss: 121154355200.0000 - val_mae: 2283.8425 - val_r2: 0.9720
Epoch 4/100
2659/2659 [==============================] - 11s 4ms/step - loss: 31084371968.0000 - mae: 721.7868 - r2: 0.9793 - val_loss: 105244377088.0000 - val_mae: 2109.1028 - val_r2: 0.9804
Epoch 5/100
2659/2659 [==============================] - 11s 4ms/step - loss: 27006093312.0000 - mae: 666.5

In [24]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 64)                19136     
                                                                 
 dense_5 (Dense)             (None, 32)                2080      
                                                                 
 dense_6 (Dense)             (None, 16)                528       
                                                                 
 dense_7 (Dense)             (None, 1)                 17        
                                                                 
Total params: 21,761
Trainable params: 21,761
Non-trainable params: 0
_________________________________________________________________


In [25]:
model.get_weights()

[array([[ 0.14507185,  0.14486305, -0.12447232, ...,  0.10942489,
          0.03480353,  0.13491724],
        [-0.10547385,  0.190359  ,  0.06718028, ...,  0.16910346,
          0.10019575,  0.19637038],
        [-0.08481486,  0.07543398,  0.14155987, ...,  0.18131527,
          0.14463028,  0.17069134],
        ...,
        [ 0.05778207,  0.09877542, -0.07548293, ...,  0.02144895,
         -0.05858679, -0.07897908],
        [-0.09556732,  0.02761653,  0.01556758, ...,  0.17240033,
          0.11458647,  0.11556623],
        [-0.06205273,  0.01194631,  0.20093086, ..., -0.02236433,
         -0.05817188, -0.12013383]], dtype=float32),
 array([ 0.01278162,  0.08735537,  0.08128688,  0.        ,  0.        ,
         0.1531018 , -0.01502975,  0.1433278 , -0.00918079,  0.25308925,
         0.10503456,  0.        ,  0.03603362, -0.08163118, -0.05889774,
         0.        ,  0.10662786,  0.1365212 ,  0.05801091, -0.07630891,
         0.10829695,  0.        ,  0.        , -0.04567743,  0.098

In [26]:
# model.load_weights('best_weights_amsgrad.h5')
# model.get_weights()

In [27]:
# score = model.evaluate(X_test, y_test)
# print(f'Deep Learning Regression Mean Absolute Error: {score[1]:.2f}')
# print(f'Deep Learning Regression R-squared Score: {score[2]:.2f}')

In [31]:
predictions = model.predict(X_test)

# Print the predicted prices
predictions

1299/1299 [==============================] - 3s 2ms/step


array([[142608.22  ],
       [  1092.0286],
       [  4483.9033],
       ...,
       [  2196.6584],
       [  6503.3086],
       [ 17830.29  ]], dtype=float32)

In [29]:
y_test

120328    145000
34901       1095
165437      4480
155406      1325
24030       1800
           ...  
155498      6700
93180        650
153338      2200
131690      6500
127733     17799
Name: price, Length: 41547, dtype: int64

In [30]:
import plotly.express as px
import pandas as pd

# create a dataframe with the training history
df_graph = pd.DataFrame(history.history)

# create line charts for loss and MAE
fig = px.line(df_graph, x=df_graph.index+1, y=['loss', 'val_loss'], labels={'x':'Epoch', 'value':'Loss'},
              title='Training and Validation Loss')
fig.update_traces(mode='lines')
fig.show()

fig = px.line(df_graph, x=df_graph.index+1, y=['mae', 'val_mae'], labels={'x':'Epoch', 'value':'MAE'},
              title='Training and Validation MAE')
fig.update_traces(mode='lines')
fig.show()